In [ ]:
import tensorflow as tf
from tensorflow import keras

# Creating a Convolutional Neural Network

### Access images located in a local directory

In [ ]:
# the os library gives you  access to the file system
import os

# listdir returns a list of the files in the directory; len() returns the number of files/items in the list
# display the number of images in the various folders

# 240 training images
print("The number of train apple images: ", len(os.listdir("fruit/train_multi/apple")))
print("The number of train banana images: ", len(os.listdir("fruit/train_multi/banana")))
print("The number of train mixed images: ", len(os.listdir("fruit/train_multi/mixed")))
print("The number of train orange images: ", len(os.listdir("fruit/train_multi/orange")))

# 60 test images
print("The number of test apple images: ", len(os.listdir("fruit/test_multi/apple")))
print("The number of test banana images: ", len(os.listdir("fruit/test_multi/banana")))
print("The number of test mixed images: ", len(os.listdir("fruit/test_multi/mixed")))
print("The number of test orange images: ", len(os.listdir("fruit/test_multi/orange")))

### Create a CNN

In [ ]:
model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 4 output neurons for multi-class output requires 'softmax' and will sum to 1. Each node will output its class' probability.
    tf.keras.layers.Dense(4, activation='softmax')
    ])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

### Normalize the data and generate labels for the images

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_data_generator = ImageDataGenerator(rescale=1/255)
validate_data_generator = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 60 using train_data_generator
train_generator = train_data_generator.flow_from_directory(
        'fruit/train_multi',  # This is the source directory for training images 
                            # that contains subdirectories(whose names will be the labels) that contain images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=60,
        # Since we use 'sparse_categorical_crossentropy' loss, we need sparse classes
        class_mode='sparse')

validate_generator = validate_data_generator.flow_from_directory(
        'fruit/test_multi',  # This is the source directory for training images 
                            # that contains subdirectories(whose names will be the labels) that contain images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=15,
        # Since we use 'sparse_categorical_crossentropy' loss, we need sparse classes
        class_mode='sparse')

In [ ]:

# Image Augmentation example

# Sometimes, depending upon the classfication problem and the images, 
# performing image augmentation can be helpful to provide variety for the model
# through random rotations, shifts, flips and zooms of the images.

# train_data_generator = ImageDataGenerator(
#                     rescale=1./255, 
#                     rotation_range=45, 
#                     width_shift_range=.15, 
#                     height_shift_range=.15, 
#                     horizontal_flip=True, 
#                     zoom_range=0.5
#                    )


### Train the CNN (using the generated labels)

In [ ]:
# fit the model using the generated labels

history = model.fit_generator(
      train_generator,
      steps_per_epoch=4,     # the ceiling of: total_number_of_images/batch_size
                             # int(np.ceil(train_generator.n / float(batch_size)))
      epochs=15,
    
      validation_data = validate_generator,
      validation_steps=4,
      verbose=1)

### Manual Validation

In [ ]:
import numpy as np
from keras .preprocessing import image


class_labels = ['apple', 'banana', 'mixed', 'orange']

# a list of images (from a directory/folder) that will be used to validate the model's predictions
validating_images = os.listdir("fruit/validating_multi_images")

# predicting images from a folder containing a set of unseen images
for file_name in validating_images:
    path = 'fruit/validating_multi_images/' + file_name
    img = image.load_img(path, target_size = (150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    images = np.vstack([x])
    
    prediction = model.predict(images)
    
    out = prediction[0]
    print(file_name, ": ", class_labels[out.argmax()])
